---
title: Overview of the event
---

In [1]:
#| hide
%load_ext autoreload
%autoreload 2

In [2]:
from sunpy.time import TimeRange
from space_analysis.missions.psp.plasma import load_data as load_data_plasma
import space_analysis.missions.dscovr as dscovr

from space_analysis.plasma.formulary import df_beta, df_Alfven_speed
from space_analysis.utils.basic import resample

import polars as pl
from beforerr.polars import pl_norm

import astropy.units as u

from datetime import timedelta

In [3]:
start = '2019-04-06T12:00'
end = '2019-04-07T12:00'

earth_start = '2019-04-09'
earth_end = '2019-04-14'

every = timedelta(minutes=4)

In [4]:
psp_timerange = TimeRange(start, end)
earth_timerange = TimeRange(earth_start, earth_end)

psp_vec_cols=[f"VEL_RTN_SUN_{i}" for i in range(3)]
magf_cols=[f"MAGF_SC_{i}" for i in range(3)]

In [5]:
def validate(timerange):
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

In [6]:
psp_plasma = load_data_plasma(trange = validate(psp_timerange), var_names=["DENS", "VEL_RTN_SUN", "TEMP", "MAGF_SC", "SUN_DIST"])

01-Feb-24 17:02:48: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/


Using LEVEL=L3


01-Feb-24 17:02:48: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190406_v04.cdf
01-Feb-24 17:02:49: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190407_v04.cdf


In [7]:
dscovr_vec_cols=[f"V_GSE_{i}" for i in range(3)]

dscovr_plasma :pl.LazyFrame = dscovr.load_data(trange = validate(earth_timerange), instrument='faraday_cup')
dscovr_mag = dscovr.load_data(trange = validate(earth_timerange), instrument='mag')

01-Feb-24 17:02:49: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h1/faraday_cup/2019/
01-Feb-24 17:02:49: File is current: /Users/zijin/data/dscovr/h1/faraday_cup/2019/dscovr_h1_fc_20190409_v14.cdf
01-Feb-24 17:02:49: File is current: /Users/zijin/data/dscovr/h1/faraday_cup/2019/dscovr_h1_fc_20190410_v14.cdf
01-Feb-24 17:02:50: File is current: /Users/zijin/data/dscovr/h1/faraday_cup/2019/dscovr_h1_fc_20190411_v14.cdf
01-Feb-24 17:02:50: File is current: /Users/zijin/data/dscovr/h1/faraday_cup/2019/dscovr_h1_fc_20190412_v14.cdf
01-Feb-24 17:02:50: File is current: /Users/zijin/data/dscovr/h1/faraday_cup/2019/dscovr_h1_fc_20190413_v14.cdf
01-Feb-24 17:02:50: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/dscovr/h0/mag/2019/
01-Feb-24 17:02:51: File is current: /Users/zijin/data/dscovr/h0/mag/2019/dscovr_h0_mag_20190409_v01.cdf
01-Feb-24 17:02:51: File is current: /Users/zijin/data/dscovr/h0/mag/2019/dscovr_h0_mag_20190410_v01.cdf
01-Feb-24 17:02:51

## Radial evolution of the coronalhole high-speed streams

The radial dependence of the proton number density, np is

$$n_p = (2.4 ± 0.1)(R/R_0)^{−(1.96±0.07)} cm^{−3}$$

$$B = (5.7 ± 0.2)(R/R_0)^{−(1.59±0.06)} nT$$

The faster decrease of the magnetic than kinetic pressure is reﬂected in the radial proton plasma beta variation

$$β_p = P_k/P_B = (0.55 ± 0.04)(R/R_0)^{(0.4±0.1)}.$$

The behaviour of the parallel proton plasma beta is similar 

$$β_{‖} = (0.37 ± 0.03)(R/R_0)(0.8±0.1)$$

In [8]:
# The radial dependence of the proton number density, np is

# $$n_p = (2.4 ± 0.1)(R/R_0)^{−(1.96±0.07)} cm^{−3}$$

# $$B = (5.7 ± 0.2)(R/R_0)^{−(1.59±0.06)} nT$$

# The faster decrease of the magnetic than kinetic pressure is reﬂected in the radial proton plasma beta variation

# $$β_p = P_k/P_B = (0.55 ± 0.04)(R/R_0)^{(0.4±0.1)}.$$

# The behaviour of the parallel proton plasma beta is similar 

# $$β_{‖} = (0.37 ± 0.03)(R/R_0)(0.8±0.1)$$

def plasma_r_evolution(
    df: pl.DataFrame,
    alpha_beta_r = 0.4,
    alpha_beta_parallel_r = 0.9,
    alpha_n = -2,
    alpha_B = -1.63,
    alpha_plasma_speed = 0,
):
    return df.with_columns(
        plasma_speed_1AU = pl.col('plasma_speed') * (1 / pl.col("distance2sun"))**alpha_plasma_speed,
        n_1AU = pl.col('n') * (1 / pl.col("distance2sun"))**alpha_n,
        B_1AU = pl.col('B') * (1 / pl.col("distance2sun"))**alpha_B,
        beta_1AU = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_r,
        beta_parallel = pl.col("beta") * (1 / pl.col("distance2sun"))**alpha_beta_parallel_r,
    )

In [9]:
km2au = u.km.to(u.AU)

In [10]:
dscovr_df = (
    dscovr_plasma.pipe(resample, every)
    .sort("time")
    .join_asof(dscovr_mag.pipe(resample, every).sort("time"), on="time")
    .with_columns(
        plasma_speed=pl_norm(dscovr_vec_cols),
    )
    .rename(
        {   
            "B1F1": "B",
            "Np": "n",
            "THERMAL_TEMP": "T",
        }
    )
    .collect()
    .pipe(df_beta, T_unit = u.K)
    .pipe(df_Alfven_speed)
    .with_columns(
        plasma_speed_over_Alfven_speed=pl.col("plasma_speed") / pl.col("Alfven_speed"),
    )
)

In [11]:

psp_plasma_r = (
    psp_plasma.pipe(resample, every)
    .with_columns(
        B=pl_norm(magf_cols),
        plasma_speed = pl_norm(psp_vec_cols),
        distance2sun = pl.col('SUN_DIST') * km2au,
    )
    .rename(
        {
            "DENS": "n",
            "TEMP": "T",
        }
    )
    .collect()
    .pipe(df_beta)
    .pipe(plasma_r_evolution)
    .pipe(df_Alfven_speed)
    .pipe(df_Alfven_speed, B = "B_1AU", n = "n_1AU", col_name = "Alfven_speed_1AU")
    .with_columns(
        plasma_speed_over_Alfven_speed = pl.col("plasma_speed") / pl.col("Alfven_speed"),
        plasma_speed_over_Alfven_speed_1AU = pl.col("plasma_speed_1AU") / pl.col("Alfven_speed_1AU"),
    )
)

In [12]:
plot = psp_plasma_r.plot(x="time", y=["plasma_speed", "beta", "Alfven_speed", "plasma_speed_over_Alfven_speed"], subplots=True, shared_axes=False).cols(1)

plot_1AU = psp_plasma_r.plot(x="time", y=["plasma_speed_1AU", "beta_1AU", "Alfven_speed_1AU", "plasma_speed_over_Alfven_speed_1AU"], subplots=True, shared_axes=False).cols(1)

plot + plot_1AU

:Layout
   .NdLayout.I  :NdLayout   [Variable]
      :Curve   [time]   (value)
   .NdLayout.II :NdLayout   [Variable]
      :Curve   [time]   (value)

In [13]:
(psp_plasma_r.plot.scatter(x="beta", y="plasma_speed_over_Alfven_speed", logx=True, logy=True) + psp_plasma_r.plot.scatter(x="beta_1AU", y="plasma_speed_over_Alfven_speed_1AU", logx=True, logy=True)).cols(1)

:Layout
   .Scatter.I  :Scatter   [beta]   (plasma_speed_over_Alfven_speed)
   .Scatter.II :Scatter   [beta_1AU]   (plasma_speed_over_Alfven_speed_1AU)

In [15]:
psp_plasma_r.plot.scatter(
    x="beta", y="plasma_speed_over_Alfven_speed", logx=True, logy=True, label='PSP'
) * psp_plasma_r.plot.scatter(
    x="beta_1AU", y="plasma_speed_over_Alfven_speed_1AU", logx=True, logy=True, label='PSP predicted at 1AU'
) * dscovr_df.plot.scatter(
    x="beta", y="plasma_speed_over_Alfven_speed", logx=True, logy=True, label='DSCOVR'
)

:Overlay
   .Scatter.PSP                  :Scatter   [beta]   (plasma_speed_over_Alfven_speed)
   .Scatter.PSP_predicted_at_1AU :Scatter   [beta_1AU]   (plasma_speed_over_Alfven_speed_1AU)
   .Scatter.DSCOVR               :Scatter   [beta]   (plasma_speed_over_Alfven_speed)